In [13]:
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import files
uploaded = files.upload()  # 여기서 "tourism_monthly_dataset.csv" 파일을 업로드하세요.

# 3. 파일명 설정 (업로드한 파일명이 다를 경우 수정)
file_name = "tourism_monthly_dataset.csv"

# 4. 파일 전체 라인을 읽어오기
with open(file_name, 'r') as f:
    lines = f.readlines()

# 5. '@data' 섹션의 인덱스 찾기 (실제 데이터가 시작되는 부분)
data_index = None
for i, line in enumerate(lines):
    if line.strip().lower() == '@data':
        data_index = i
        break

Saving tourism_monthly_dataset.csv to tourism_monthly_dataset (2).csv


In [14]:
# data_index가 None인지 확인 후, 데이터 부분(@data) 이후부터 파싱 시작
if data_index is None:
  print("데이터 부분(@data)을 찾을 수 없습니다.")
else:
  data_lines = lines[data_index+1:]

  records = []
  for line in data_lines:
    line = line.strip()
    if not line:
      continue
    parts = line.split(':')
    if len(parts) < 3:
      continue
    # series_id에서 불필요한 따옴표 제거
    series_id = parts[0].replace('"', '')
    start_date = parts[1]
    values_str = ":".join(parts[2:])

    # 숫자 변환 없이 값들을 문자열 리스트로 저장하며 불필요한 따옴표 제거
    values = [x.strip().replace('"', '') for x in values_str.split(',') if x.strip()]
    records.append({'series_id': series_id, 'start_date': start_date, 'values': values})

  # 파싱한 메타데이터를 DataFrame으로 변환
  df_series = pd.DataFrame(records)
  print("시계열 데이터 (메타정보) 미리보기:")
  display(df_series.head())

  # 모든 시계열의 최대 길이 계산
  max_len = max(len(r['values']) for r in records)

  # 확장 DataFrame 생성: series_id, start_date 및 각 월별 값(t1, t2, ...) 추가
  df_expanded = pd.DataFrame()
  df_expanded['series_id'] = [r['series_id'] for r in records]
  df_expanded['start_date'] = [r['start_date'] for r in records]
  for i in range(max_len):
      col = []
      for r in records:
          if i < len(r['values']):
              col.append(r['values'][i])
          else:
              col.append(None)
      df_expanded[f't{i+1}'] = col

  # 열 이름 변경: t1, t2, ... --> Month1, Month2, ...
  rename_dict = {}
  for col in df_expanded.columns:
      if col.startswith('t'):
          rename_dict[col] = 'Month' + col[1:]
  df_expanded.rename(columns=rename_dict, inplace=True)

  print("확장된 시계열 데이터 테이블 미리보기:")
  display(df_expanded.head())


시계열 데이터 (메타정보) 미리보기:


,series_id,start_date,values
0,T1,1979-01-01 00-00-00,"[1149.87, 1053.8002, 1388.8798, 1783.3702, 192..."
1,T2,1979-01-01 00-00-00,"[65072.37, 48612.2, 58452.59, 57033.97, 71498...."
2,T3,1985-01-01 00-00-00,"[37047, 32836, 51094, 52629, 56884, 52955, 560..."
3,T4,1985-01-01 00-00-00,"[7611, 6859, 9247, 10924, 9064, 7417, 10467, 7..."
4,T5,1985-01-01 00-00-00,"[27693, 15075, 22602, 23530, 26817, 22696, 204..."


확장된 시계열 데이터 테이블 미리보기:


<ipython-input-14-cf39c5e09e6b>:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_expanded[f't{i+1}'] = col
<ipython-input-14-cf39c5e09e6b>:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_expanded[f't{i+1}'] = col
<ipython-input-14-cf39c5e09e6b>:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

,series_id,start_date,Month1,Month2,Month3,Month4,Month5,Month6,Month7,Month8,...,Month324,Month325,Month326,Month327,Month328,Month329,Month330,Month331,Month332,Month333
0,T1,1979-01-01 00-00-00,1149.87,1053.8002,1388.8798,1783.3702,1921.0252,2704.9449,4184.4135,4148.3542,...,None,None,None,None,None,None,None,None,None,None
1,T2,1979-01-01 00-00-00,65072.37,48612.2,58452.59,57033.97,71498.95,79187.11,101896.1,115971.8,...,None,None,None,None,None,None,None,None,None,None
2,T3,1985-01-01 00-00-00,37047,32836,51094,52629,56884,52955,56012,50462,...,None,None,None,None,None,None,None,None,None,None
3,T4,1985-01-01 00-00-00,7611,6859,9247,10924,9064,7417,10467,7921,...,None,None,None,None,None,None,None,None,None,None
4,T5,1985-01-01 00-00-00,27693,15075,22602,23530,26817,22696,20433,24152,...,None,None,None,None,None,None,None,None,None,None


In [15]:
# 확장 DataFrame(df_expanded)이 이미 생성되었다고 가정합니다.
output_filename = "expanded_tourism_dataset.csv"
df_expanded.to_csv(output_filename, index=False)
print("CSV 파일이 저장되었습니다:", output_filename)

# 저장된 CSV 파일을 로컬로 다운로드합니다.
from google.colab import files
files.download(output_filename)

CSV 파일이 저장되었습니다: expanded_tourism_dataset.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>